# 1. Las variables que se van a cargar

In [0]:
import os
MODEL_NAME = 'MILEtf'
TRAINING_DIR = 'MILEtf_trained'
os.environ['MODEL_NAME'] = MODEL_NAME
os.environ['TRAINING_DIR'] = TRAINING_DIR 

La carpeta tiene que estar en el raiz donde se abrió Jupyter

In [0]:
%%bash
find ${MODEL_NAME}

## 1.1. Find absolute paths to your data
Note the absolute paths below. /content is mapped in Datalab to where the home icon takes you

In [0]:
%%bash
echo "Working Directory: ${PWD}"
echo "Head of taxi-train.csv"
head -1 $PWD/${MODEL_NAME}/data/taxi-train.csv
echo "Head of taxi-valid.csv"
head -1 $PWD/${MODEL_NAME}/data/taxi-valid.csv

# 2. Running localmente usando Python
Clean model training dir/output dir

In [0]:
%%bash
# This is so that the trained model is started fresh each time. However, this needs to be done before 
# tensorboard is started
rm -rf $PWD/${MODEL_NAME}/${TRAINING_DIR}

In [0]:
%reload_ext tensorboard.notebook
!kill 26773

In [0]:
# tensorboard is started
%load_ext tensorboard.notebook
logs_base_dir = "./MILEtf/MILEtf_trained"
os.makedirs(logs_base_dir, exist_ok=True)
%tensorboard --logdir {logs_base_dir}

In [0]:
%%bash
# Setup python so it sees the task module which controls the model.py
export PYTHONPATH=${PYTHONPATH}:${PWD}/${MODEL_NAME}
# Currently set for python 2.  To run with python 3 
#    1.  Replace 'python' with 'python3' in the following command
#    2.  Edit trainer/task.py to reflect proper module import method 
python -m trainer.task \
   --train_data_paths="${PWD}/${MODEL_NAME}/data/taxi-train*" \
   --eval_data_paths=${PWD}/${MODEL_NAME}/data/taxi-valid.csv  \
   --output_dir=${PWD}/${MODEL_NAME}/${TRAINING_DIR} \
   --train_steps=1000 --job-dir=./tmp

Modelo exportado generado

In [0]:
%%bash
ls $PWD/${MODEL_NAME}/${TRAINING_DIR}/export/exporter/

# 3. Prediccion usando gcloud
Archivo de prueba

In [0]:
%%writefile ./{MODEL_NAME}/test.json
{"pickuplon": -73.885262,"pickuplat": 40.773008,"dropofflon": -73.987232,"dropofflat": 40.732403,"passengers": 2}

Prediccion local usando gcloud

In [0]:
%%bash
# Esta linea es porque el pyc de esa ruta esta compilado para python2
rm -rf /usr/local/Caskroom/google-cloud-sdk/latest/google-cloud-sdk/lib/googlecloudsdk/command_lib/ml_engine/local_predict.pyc
# This model dir is the model exported after training and is used for prediction
#
model_dir=$(ls ${PWD}/${MODEL_NAME}/${TRAINING_DIR}/export/exporter | tail -1)
# predict using the trained model
gcloud ml-engine local predict  \
    --model-dir=${PWD}/${MODEL_NAME}/${TRAINING_DIR}/export/exporter/${model_dir} \
    --json-instances=./${MODEL_NAME}/test.json

# 4. Running localmente usando gcloud

In [0]:
%%bash
rm -rf $PWD/${MODEL_NAME}/${TRAINING_DIR}
# Use Cloud Machine Learning Engine to train the model in local file system
gcloud ml-engine local train \
   --module-name=trainer.task \
   --package-path=${PWD}/${MODEL_NAME}/trainer \
   -- \
   --train_data_paths=${PWD}/${MODEL_NAME}/data/taxi-train.csv \
   --eval_data_paths=${PWD}/${MODEL_NAME}/data/taxi-valid.csv  \
   --train_steps=1000 \
   --output_dir=${PWD}/${MODEL_NAME}/${TRAINING_DIR} 

# 5. Enviar trabajo a MILE
## 5.1. Creacion de variables

In [0]:
# For Python Code
PROJECT = 'hdlaml' # REPLACE WITH YOUR PROJECT ID
BUCKET = 'hdlaml_taxi' # REPLACE WITH YOUR BUCKET NAME
REGION = 'us-central1' # REPLACE WITH YOUR BUCKET REGION e.g. us-central1
MODEL_VERSION = 'v2' # Model Version

# For Bash Code
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION
os.environ['MODEL_VERSION'] = MODEL_VERSION
os.environ['TRAINING_DIR'] = TRAINING_DIR 
os.environ['TFVERSION'] = '1.8'  # Tensorflow version

In [0]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Iniciar sesion en GCP

In [0]:
%%bash
#gcloud auth application-default login # No sirve para hacer cambios
#gcloud config set project PROJECT_ID
gcloud auth login # Sirve para todo

Creacion de un bucket

In [0]:
%%bash
# The bucket needs to exist for the gsutil commands in next cell to work
gsutil mb -p ${PROJECT} gs://${BUCKET}

Allow the Cloud ML Engine service account to read/write to the bucket containing training data.

In [0]:
%%bash
# This command will fail if the Cloud Machine Learning Engine API is not enabled using the link above.
echo "Getting the service account email associated with the Cloud Machine Learning Engine API"

AUTH_TOKEN=$(gcloud auth print-access-token)
SVC_ACCOUNT=$(curl -X GET -H "Content-Type: application/json" \
    -H "Authorization: Bearer $AUTH_TOKEN" \
    https://ml.googleapis.com/v1/projects/${PROJECT}:getConfig \
    | python -c "import json; import sys; response = json.load(sys.stdin); \
    print (response['serviceAccount'])")  # If this command fails, the Cloud Machine Learning Engine API has not been enabled above.

echo "Authorizing the Cloud ML Service account $SVC_ACCOUNT to access files in $BUCKET"
gsutil -m defacl ch -u $SVC_ACCOUNT:R gs://$BUCKET   
gsutil -m acl ch -u $SVC_ACCOUNT:R -r gs://$BUCKET   # error message (if bucket is empty) can be ignored.  
gsutil -m acl ch -u $SVC_ACCOUNT:W gs://$BUCKET     

First copy the training data to the cloud. Then, launch a training job.

In [0]:
%%bash
# Clear Cloud Storage bucket and copy the CSV files to Cloud Storage bucket
echo $BUCKET
gsutil -m rm -rf gs://${BUCKET}/${MODEL_NAME}/smallinput/
gsutil -m cp ${PWD}/${MODEL_NAME}/data/*.csv gs://${BUCKET}/${MODEL_NAME}/smallinput/

Enviar el trabajo para que entrene.

<b>Use the Cloud Console link to monitor the job and do NOT proceed until the job is done.<b>

In [0]:
%%bash
OUTDIR=gs://${BUCKET}/${MODEL_NAME}/smallinput/${TRAINING_DIR}
JOBNAME=${MODEL_NAME}_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
# Clear the Cloud Storage Bucket used for the training job
gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
   --region=$REGION \
   --module-name=trainer.task \
   --package-path=${PWD}/${MODEL_NAME}/trainer \
   --job-dir=$OUTDIR \
   --staging-bucket=gs://$BUCKET \
   --scale-tier=BASIC \
   --runtime-version=$TFVERSION \
   -- \
   --train_data_paths="gs://${BUCKET}/${MODEL_NAME}/smallinput/taxi-train*" \
   --eval_data_paths="gs://${BUCKET}/${MODEL_NAME}/smallinput/taxi-valid*"  \
   --output_dir=$OUTDIR \
   --train_steps=10000

## 5.2. Deploy model

Find out the actual name of the subdirectory where the model is stored and use it to deploy the model.  Deploying model will take up to <b>5 minutes</b>.

In [0]:
%%bash
gsutil ls gs://${BUCKET}/${MODEL_NAME}/smallinput/${TRAINING_DIR}/export/exporter

#### Deploy model : step 1 - remove version info 
Before an existing cloud model can be removed, it must have any version info removed.  If an existing model does not exist, this command will generate an error but that is ok.

In [0]:
%%bash
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/${MODEL_NAME}/smallinput/${TRAINING_DIR}/export/exporter | tail -1)

echo "MODEL_LOCATION = ${MODEL_LOCATION}"

gcloud ml-engine versions delete ${MODEL_VERSION} --model ${MODEL_NAME}

#### Deploy model: step 2 - remove existing model
Now that the version info is removed from an existing model, the actual model can be removed.  If an existing model is not deployed, this command will generate an error but that is ok.  It just means the model with the given name is not deployed.

In [0]:
%%bash
gcloud ml-engine models delete ${MODEL_NAME}

#### Deploy model: step 3 - deploy new model

In [0]:
%%bash
gcloud ml-engine models create ${MODEL_NAME} --regions $REGION

#### Deploy model: step 4 - add version info to the new model

In [0]:
%%bash
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/${MODEL_NAME}/smallinput/${TRAINING_DIR}/export/exporter | tail -1)

echo "MODEL_LOCATION = ${MODEL_LOCATION}"

gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION} --runtime-version $TFVERSION

## Prediction

In [0]:
%%bash
gcloud ml-engine predict --model=${MODEL_NAME} --version=${MODEL_VERSION} --json-instances=./${MODEL_NAME}/test.json

In [0]:
from googleapiclient import discovery
from oauth2client.client import GoogleCredentials
import json

credentials = GoogleCredentials.get_application_default()
api = discovery.build('ml', 'v1', credentials=credentials,
            discoveryServiceUrl='https://storage.googleapis.com/cloud-ml/discovery/ml_v1_discovery.json')

request_data = {'instances':
  [
      {
        'pickuplon': -73.885262,
        'pickuplat': 40.773008,
        'dropofflon': -73.987232,
        'dropofflat': 40.732403,
        'passengers': 2,
      }
  ]
}

parent = 'projects/%s/models/%s/versions/%s' % (PROJECT, MODEL_NAME, MODEL_VERSION)
response = api.projects().predict(body=request_data, name=parent).execute()
print(response)

### Clean-up

#### Delete Model : step 1 - remove version info 
Before an existing cloud model can be removed, it must have any version info removed.  

In [0]:
%%bash
gcloud ml-engine versions delete ${MODEL_VERSION} --model ${MODEL_NAME}

#### Delete model: step 2 - remove existing model
Now that the version info is removed from an existing model, the actual model can be removed.  

In [0]:
%%bash
gcloud ml-engine models delete ${MODEL_NAME}

Copyright 2016 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License